In [20]:
from Env import AimsunEnv
from DQN import ddqn_framework as ddqn
from DQN import mlp_framework as nn
import numpy as np
import pickle
import itertools

# env setup
intx_action = np.arange(-20, 21, 5)
action_space = list(itertools.product(intx_action, intx_action))
env = AimsunEnv(action_space)

# model config
configuration = ddqn.trainer_config(app_name='AimsunNext',
                                # BUFFER_SIZE = 50e3,
                                BUFFER_SIZE = 50000,
                                # STEPS_PER_EPISODE = 500,
                                STEPS_PER_EPISODE=500,
                                MAX_STEPS = 9999999,
                                # UPDATE_TARGET_STEPS = 1000,
                                UPDATE_TARGET_STEPS=1000,
                                BATCH_SIZE = 32,
                                GAMMA = 0.9,
                                EXPLORATION = 5000,
                                E_MIN = 0.1,
                                priority = True,
                                # alpha = 0.001,
                                alpha = 0.01,
                                epsilon = 0.1
                                )

# online model
A1 = nn.layer(env.get_state_size(), nodes=512)
A2 = nn.layer(512, 512)
A3 = nn.layer(512, 512)
A4 = nn.layer(512, 512)
AOUT = nn.layer(512, env.get_action_size())
AOUT.f = nn.f_iden

# target model
L1 = nn.layer(env.get_state_size(), nodes=512)
L2 = nn.layer(512, 512)
L3 = nn.layer(512, 512)
L4 = nn.layer(512, 512)
LOUT = nn.layer(512, env.get_action_size())
LOUT.f = nn.f_iden

L1.w = np.copy(A1.w)
L2.w = np.copy(A2.w)
L3.w = np.copy(A3.w)
L4.w = np.copy(A4.w)
LOUT.w = np.copy(AOUT.w)

onlineNet = nn.mlp([A1, A2, A3, A4, AOUT])
onlineNet.erf = nn.log_cosh  # cost in prev aimsun_dqn
targetNet = nn.mlp([L1, L2, L3, L4, LOUT])
targetNet.erf = nn.log_cosh  # cost in prev aimsun_dqn

ddqn_model = ddqn.trainer(onlineNet,targetNet,configuration, env)
ddqn_model.load_model(False)
ddqn_model.train(log=True)

array([[-0.96471424,  1.71370023,  0.88005325, -0.35750282, -1.707765  ,
        -0.2233411 ,  1.24388639, -1.0503453 ,  1.42182763,  0.06624914,
         0.22430001, -0.83322966,  0.58717292,  1.59452717,  0.43566004,
         1.70898366]])

In [27]:
max(1, 2)

2